# **Load and Profile the Excel Data**

In [66]:
import re, pandas as pd
from datetime import datetime
import numpy as np
from collections import Counter


def is_datetime(value):
    try:
        datetime.strptime(str(value), "%Y-%m-%d %H:%M:%S")
        return True
    except ValueError:
        return False

def is_time(value):
    try:
        datetime.strptime(str(value), "%H:%M:%S")
        return True
    except ValueError:
        return False

def is_date(value):
    try:
        datetime.strptime(str(value), "%Y-%m-%d")
        return True
    except ValueError:
        return False
    
def is_boolean(value):
    return str(value).lower() in {'true', 'false', '1', '0', 'yes', 'no'}

def is_email(value):
    return bool(re.match(r"^[\w\.-]+@[\w\.-]+\.\w+$", str(value)))

def is_url(value):
    return bool(re.match(r"^https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+$", str(value)))

def is_currency(value):
    return bool(re.match(r"^\$?(\d+(\.\d{2})?)$", str(value)))

def is_phone(value):
    return bool(re.match(r"^\+?[\d\s\-\(\)]{7,15}$", str(value)))
    
    

def detect_data_type(values):
    sample_values = values[:]
    
    if all(isinstance(val, (int, float)) and not pd.isna(val) for val in sample_values):
        return 'numeric'
    
    elif all(is_datetime(val) for val in sample_values):
        return 'datetime'
    
    elif all(is_time(val) for val in sample_values):
        return 'time'
    
    elif all(is_date(val) for val in sample_values):
        return 'date'
    
    elif all(is_boolean(val) for val in sample_values):
        return 'boolean'
    
    elif all(is_email(val) for val in sample_values):
        return 'email'
    
    elif all(is_url(val) for val in sample_values):
        return 'url'
    
    elif all(is_phone(val) for val in sample_values):
        return 'phone'
    
    else:
        return 'string'

def analyze_column_data(values, data_type):
    if data_type == 'numeric':
        numbers = [val for val in values if isinstance(val, (int, float))]
        return {
                'mean': np.mean(numbers),
                'median': np.median(numbers),
                'stdDev': np.std(numbers, ddof=1),
                'min': min(numbers),
                'max': max(numbers),
                'count': len(numbers),
            }
        
    elif data_type == 'datetime':
        datetimes = [datetime.strptime(val, "%Y-%m-%d %H:%M:%S") for val in values if isinstance(val, str)]
        return {
            'minDatetime': min(datetimes).strftime("%Y-%m-%d %H:%M:%S"),
            'maxDatetime': max(datetimes).strftime("%Y-%m-%d %H:%M:%S"),
        }
    
    elif data_type == 'time':
        times = [datetime.strptime(val, "%H:%M:%S").time() for val in values if isinstance(val, str)]
        return {
            'minTime': min(times).strftime("%H:%M:%S"),
            'maxTime': max(times).strftime("%H:%M:%S"),
        }
        
    elif data_type == 'date':
        dates = [datetime.strptime(val, "%Y-%m-%d") for val in values if is_date(val)]
        return {
            'minDate': min(dates).strftime("%Y-%m-%d"),
            'maxDate': max(dates).strftime("%Y-%m-%d"),
        }
        
    elif data_type == 'boolean':
        booleans = [val for val in values if isinstance(val, bool)]
        return {
            'trueCount': booleans.count(True),
            'falseCount': booleans.count(False),
            'uniqueValues': [True, False],
        }
    
    elif data_type == 'url':
        urls = [val for val in values if is_url(val)]
        return {
            'urlCount': len(urls),
            'uniqueUrls': list(set(urls)),
        }

    elif data_type == 'email':
        emails = [val for val in values if is_email(val)]
        return {
            'emailCount': len(emails),
            'uniqueEmails': list(set(emails)),
        }
    
    elif data_type == 'phone':
        phones = [val for val in values if is_phone(val)]
        return {
            'phoneCount': len(phones),
            'uniquePhones': list(set(phones)),
        }
        
    else:
        unique_values = set(values)
        value_counts = {val: values.count(val) for val in unique_values}
        text_values = [str(val) for val in values if isinstance(val, str)]
        lengths = [len(val) for val in text_values]
        most_common_10_words = Counter(" ".join(text_values).split()).most_common(10)
        
        return {
            'textCount': len(text_values),
            'averageLength': np.mean(lengths) if lengths else 0,
            'mostCommonTenWords': most_common_10_words,
            'uniqueValues': list(unique_values),
            'uniqueCount': len(unique_values),
            'valueCounts': value_counts,
        }
        
def detect_data_orientation(df):
    
    row_sample_list = df.iloc[:, 0].tolist() 
    
    columns = df.columns.tolist()
    
    if row_sample_list == columns:
        return 1
    else:
        return 0
        
def load_and_profile_data(file_path):
    df = pd.read_excel(file_path)
    df = df.fillna(value=pd.NA) 
    
    if detect_data_orientation(df):
        df = df.transpose()

    data_profile = {}
    for column in df.columns:
        values = df[column].dropna().tolist()
        data_type = detect_data_type(values)
        analysis = analyze_column_data(values, data_type)
        data_profile[column] = {
            'dataType': data_type,
            'analysis': analysis,
        }

    return {'jsonData': df.to_dict(orient='records'), 'dataProfile': data_profile}


In [74]:
result = load_and_profile_data('/Users/salehahmedshafin/Downloads/Jupyter/input_content/Realcomm Program Details  (1).xlsx')
print(result['dataProfile'])

{'Event': {'dataType': 'string', 'analysis': {'textCount': 150, 'averageLength': 11.66, 'mostCommonTenWords': [('2024', 150), ('Realcomm', 83), ('IBcon', 67)], 'uniqueValues': ['Realcomm 2024', 'IBcon 2024'], 'uniqueCount': 2, 'valueCounts': {'Realcomm 2024': 83, 'IBcon 2024': 67}}}, 'Track': {'dataType': 'string', 'analysis': {'textCount': 150, 'averageLength': 27.04, 'mostCommonTenWords': [('PRECON:', 75), ('Forum', 40), ('Buildings', 35), ('–', 21), ('Smarter', 18), ('Smarter,', 15), ('Connected', 15), ('General', 14), ('Smart', 13), ('CRE', 12)], 'uniqueValues': ['PRECON: BuildingsAI Symposium', 'PRECON: Sustainable Impact Forum', 'Smarter Buildings – Lobby and Amenities', 'Investment Management', 'Open Forum Discussions by Sector', 'REthink Work', 'Smarter Buildings – Data / AI', 'Smarter Buildings – Op Tech Briefings', 'Cyber / Privacy', 'Executive Leadership – Moving the Needle', 'Low Voltage Buildings', 'General Session', 'PRECON: CIO Roundtable', 'PRECON: CRE Cybersecurity For

# **Generate Summaries and Potential Questions Using OpenAI API**

In [75]:
import openai
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY']= 'sk-...'
client = OpenAI()

def generate_text_with_llm(prompt):
    
    messages=[
        
        {"role": "system", "content": prompt}
        
    ]
    
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    
    result = completion.choices[0].message.content.strip()
    return result

def generate_summaries_and_questions(data_profile):
    summaries = []
    questions = []

    for column, profile in data_profile.items():
        data_type = profile.get("dataType")
        analysis = profile.get("analysis")

        if not analysis:
            continue

        summary_prompt = ""
        question_prompt = ""

        if data_type == "numeric":
            summary_prompt = (
                f"Summarize the numeric data for the column **{column}** with mean, "
                f"sum, min, stdDev and max. The values are: {analysis}."
            )
            question_prompt = f"Generate possible user questions and answers about the numeric data in the column **{column}** according to the values: **{analysis}**."
            
        elif data_type == "string":
            unique_values_preview = ", ".join(str(val) for val in analysis["uniqueValues"][:])
            summary_prompt = (
                f"Summarize the categorical data for the column **{column}** which has the following unique values: "
                f"{unique_values_preview}"
            )
            question_prompt = f"Generate possible user questions and answers about the categories in the column **{column}** according to the: **{unique_values_preview}**."
            
        elif data_type == "date":
            
            summary_prompt = (
                f"Summarize the date range for the column **{column}** with min date and max date. "
                f"The values are: {analysis}."
            )
            question_prompt = f"Generate possible user questions and answers about the dates in the column **{column}** according to the: **{analysis}**."
        else:
            continue


        summary = generate_text_with_llm(summary_prompt)
        summaries.append(summary)


        generated_questions = generate_text_with_llm(question_prompt)
        questions.append(generated_questions)

    return {"summaries": summaries, "questions": questions}


result = generate_summaries_and_questions(result['dataProfile'])
print(result["summaries"])
print(result["questions"])


['- Realcomm 2024: Appears multiple times in the dataset as an event.\n- IBcon 2024: Also appears multiple times in the dataset as an event.', 'The column **Track** contains categorical data with a variety of unique values including different symposiums, forums, discussions, and sessions related to various topics within the realm of smart buildings, data analytics, cybersecurity, leadership, process automation, and more. The data includes tracks such as sustainability, investment management, cybersecurity, data analytics, AI, PropTech, automation, and in-building wireless among others.', 'The categorical data in the **Session Title** column includes a wide range of topics related to real estate, technology, innovation, cybersecurity, sustainability, and leadership. Some of the unique session titles include "The Talent Gap," "Revolutionizing Spaces: What’s Really Driving Meaningful Occupant Experience in the Digital Age?," "Integrated Digital Solutions in Real Estate: A Journey from Des

In [78]:
len(result["summaries"])

24

# **Compute Embeddings**

In [71]:
from langchain_openai import OpenAIEmbeddings

summaries = ''
questions = ''
for i in range(len(result["summaries"])):
  summaries += result["summaries"][i]
  questions += result["questions"][i]

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
two_vectors = embeddings.embed_documents([summaries,questions])

In [72]:
len(two_vectors[1])

1536

# **Store in Zilliz**

In [ ]:
import tiktoken
from pymilvus import MilvusClient
from langchain_openai import OpenAIEmbeddings

ZILLIZ_CLOUD_URI = "https://in03-d13b56d6e07779a.serverless.gcp-us-west1.cloud.zilliz.com"
ZILLIZ_TOKEN = "7422884e78d5250d8d168652e7d8b8a823d82ef6e4a29e929a96d3556d31a5d5bf36b34c26f3e8af45f98e5e4caaded0e0f959f7"
COLLECTION_NAME = "dev_collection"

milvus_client = MilvusClient(
        uri=ZILLIZ_CLOUD_URI,
        token=ZILLIZ_TOKEN
    )

def token_count(summaries, questions):
    text = summaries + questions
    encoding = tiktoken.encoding_for_model("gpt-4")
    tokens = encoding.encode(text)
    num_tokens = len(tokens)
    
    return num_tokens 

def text_embedding(summaries: str, questions: str):
    text = str(summaries) + str(questions)
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )
    vector = embeddings.embed_query(text)
    return vector


def insert_data(summaries, questions):
    
    data_batch = []
        
    for i in range(len(summaries)): 
        data_batch.append({
                'summaries': summaries[i],
                'questionsAndAnswers':questions[i],
                'tokens': token_count(summaries[i], questions[i]),
                'vectors': text_embedding(summaries[i], questions[i])
            })
        
    milvus_client.insert(
        collection_name= COLLECTION_NAME,
        data=data_batch
    )
                
    return "File processed and stored successfully in ZILLIZ!"


insert_data(result["summaries"], result["questions"])

'File processed and stored successfully in ZILLIZ!'

In [95]:
from langchain_openai import OpenAIEmbeddings

def query_data(query_text: str, top_k: int = 5):
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small"
    )
    query_vector = embeddings.embed_query(query_text)
    
    results = milvus_client.search(
        collection_name=COLLECTION_NAME,
        data=[query_vector], 
        output_fields=["summaries", "questionsAndAnswers"], 
        limit=top_k 
    )
    
    parsed_results = []
    for result in results[0]: 
        parsed_results.append({
            "summaries": result["entity"]["summaries"],
            "questionsAndAnswers": result["entity"].get("questionsAndAnswers", None), 
            "score": result["distance"] 
        })

    
    return parsed_results

query_text = "how many speaker of 9th Annual Golf Outing & Charity Event season?"
search_results = query_data(query_text)
print(search_results)


[{'summaries': 'The column **Location** contains data for various unique values such as:\n- 2nd Floor Foyer + Landing\n- Room 413\n- Main Stage\n- 2nd Floor Foyer\n- TPC Tampa Bay\n- Room 301\n- Room 411\n- Room 309\n- Room 307\n- Room 303\n- Room 305\n- Ballroom A.', 'questionsAndAnswers': '**2nd Floor Foyer + Landing**\nQ: Where can I find the networking area?\nA: The networking area is located on the 2nd Floor Foyer + Landing.\n\nQ: Are there restrooms on the second floor?\nA: Yes, you can find restrooms on the 2nd Floor Foyer.\n\n**Room 413**\nQ: What events are taking place in Room 413?\nA: Room 413 is hosting the coding workshop and the AI panel discussion today.\n\nQ: How can I access Room 413?\nA: Room 413 is located on the fourth floor.\n\n**Main Stage**\nQ: When is the keynote speaker on the Main Stage?\nA: The keynote speaker will be on the Main Stage at 10:00 AM.\n\nQ: Can I watch the presentations on the Main Stage online?\nA: Yes, the presentations on the Main Stage will 